In [70]:
import os # package for using operating system
from dotenv import load_dotenv
from snowflake.snowpark.session import Session # package for building and using Snowflake sessions
# from snowflake.cortex import Complete
# from transformers import AutoTokenizer # package to select the fitting tokenizer for a pretrained model
# from huggingface_hub import login # package for login and identifying to Huggingface
import pandas as pd # package for data manipulation
import json # package to work with .json
import math # package for mathematical operations

In [71]:
with open('../data/input_readme_data/openstack_openstack.json', 'r') as f:
    loaded_data = json.load(f)

In [72]:
repo_owner = loaded_data['repo_owner']
repo_name = loaded_data['repo_name']
input_txt = loaded_data['source_code_cleaned_comments']
cnt_input_txt = len(input_txt)

In [73]:
def write_summary_prompt(repo_name, input_txt):
    '''
    Function which writes the summary prompt for a GitHub repository.

    Args:
        repo_name: The name of the GitHub repository.
        input_txt: The source code of the GitHup repository as string.

    Return:
        prompt_summary
    '''
    prompt_summary = f'''
        You are acting as a software development expert for the following GitHub repository "{repo_name}".
        Your task is to summarize the given source code string "{input_txt}" in natural language, so a specialist is able to understand
        the purpose of the repository.
        Identify its purpose, key functions, main components and dependencies. Focus on the overall architecture and structure 
        rather than line-by-line details. Do not add any recommendations or improvement suggestions, but concentrate on the summary. 
        Present the summary in a clear and concise language.
        You are not allowed to add any small talk. 
    ''' 

    return prompt_summary

In [74]:
summary_prompt = write_summary_prompt(repo_name, input_txt)

In [75]:
len(summary_prompt)

2126

In [76]:
# load .env file
load_dotenv(override=True)

# set up connection parameters for Snowflake connection
connection_params = {
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_USER_PASSWORD'],
    "role": 'ACCOUNTADMIN',
    "warehouse": 'COMPUTE_WH',
    'paramstyle': 'qmark'
}

# build Snowflake session with connection parameters
snowflake_session = Session.builder.configs(connection_params).create()
print('Snowflake sessions is build.')
print('---------------------------------------------')

Snowflake sessions is build.
---------------------------------------------


In [77]:
#model = 'llama3.1-8b'
#model = 'mixtral-8x7b'
model = 'jamba-1.5-mini'
model_summary_params = {
   'temperature': 0, # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex --> Internetrecherche hat keine anderen Empfehlungen ergeben
   # 'top_p': # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex
    'max_tokens': 1000
}


In [78]:
query = f"""
            SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS(
                ?,
                ?
            ) AS response
        """

In [79]:
response = snowflake_session.sql(query, params=[model, summary_prompt]).collect()
# the number of tokens is lower as the acutally processed tokens because count_tokens does not account the tokens
# which are automatically added to the beginning of the the input text https://docs.snowflake.com/en/sql-reference/functions/count_tokens-snowflake-cortex

In [80]:
print(response)

[Row(RESPONSE=592)]


In [81]:
input_tokens = response[0]

In [82]:
input_tokens[0]

592

In [83]:
# estimation for number of characters which fits in 1 token
len(summary_prompt) / input_tokens[0]

3.5912162162162162

In [84]:
int(len(summary_prompt) / 4)

531

In [85]:
snowflake_session.close()